In [1]:
!pip install lightautoml

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.6/399.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.0/216.0 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 104.9 MB/s eta 0:00:0000:010:01
  Created wheel for json2html: filename=json2html-1.3.0-py3-none-any.whl size=7591 sha256=6d4c0dca551ebcd6a4359eec22637ed79063590c52283405adc1ea7acf57ce32
  Stored in directory: /root/.cache/pip/wheels/e0/d8/b3/6f83a04ab0ec00e691de794d108286bb0f8bcdf4ade19afb57
Successfully built json2html
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.14.3
    Uninstalling statsmodels-0.14.3:
      Successfully uninstalled statsmodels-0.14.3


In [2]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import FunctionTransformer, Pipeline
from sklearn.model_selection import train_test_split

import torch
from lightautoml.automl.presets.tabular_presets import (
    TabularAutoML,
    TabularUtilizedAutoML,
)
from lightautoml.tasks import Task

from sklearn.metrics import mean_squared_log_error

In [3]:
df = pd.read_csv("/kaggle/input/compete/train.csv")

In [4]:
df.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1200000 non-null  int64  
 1   Age                   1181295 non-null  float64
 2   Gender                1200000 non-null  object 
 3   Annual Income         1155051 non-null  float64
 4   Marital Status        1181471 non-null  object 
 5   Number of Dependents  1090328 non-null  float64
 6   Education Level       1200000 non-null  object 
 7   Occupation            841925 non-null   object 
 8   Health Score          1125924 non-null  float64
 9   Location              1200000 non-null  object 
 10  Policy Type           1200000 non-null  object 
 11  Previous Claims       835971 non-null   float64
 12  Vehicle Age           1199994 non-null  float64
 13  Credit Score          1062118 non-null  float64
 14  Insurance Duration    1199999 non-

In [6]:
df.describe()

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,1.200000e+06,1.181295e+06,1.155051e+06,1.090328e+06,1.125924e+06,835971.000000,1.199994e+06,1.062118e+06,1.199999e+06,1.200000e+06
mean,5.999995e+05,4.114556e+01,3.274522e+04,2.009934e+00,2.561391e+01,1.002689,9.569889e+00,5.929244e+02,5.018219e+00,1.102545e+03
std,3.464103e+05,1.353995e+01,3.217951e+04,1.417338e+00,1.220346e+01,0.982840,5.776189e+00,1.499819e+02,2.594331e+00,8.649989e+02
min,0.000000e+00,1.800000e+01,1.000000e+00,0.000000e+00,2.012237e+00,0.000000,0.000000e+00,3.000000e+02,1.000000e+00,2.000000e+01
25%,2.999998e+05,3.000000e+01,8.001000e+03,1.000000e+00,1.591896e+01,0.000000,5.000000e+00,4.680000e+02,3.000000e+00,5.140000e+02
50%,5.999995e+05,4.100000e+01,2.391100e+04,2.000000e+00,2.457865e+01,1.000000,1.000000e+01,5.950000e+02,5.000000e+00,8.720000e+02
75%,8.999992e+05,5.300000e+01,4.463400e+04,3.000000e+00,3.452721e+01,2.000000,1.500000e+01,7.210000e+02,7.000000e+00,1.509000e+03
max,1.199999e+06,6.400000e+01,1.499970e+05,4.000000e+00,5.897591e+01,9.000000,1.900000e+01,8.490000e+02,9.000000e+00,4.999000e+03


In [7]:
def date(df):
    df["Policy Start Date"] = pd.to_datetime(df["Policy Start Date"])
    df["Year"] = df["Policy Start Date"].dt.year
    df["Day"] = df["Policy Start Date"].dt.day
    df["Month"] = df["Policy Start Date"].dt.month
    df["Month_name"] = df["Policy Start Date"].dt.month_name()
    df["Day_of_week"] = df["Policy Start Date"].dt.day_name()
    df["Week"] = df["Policy Start Date"].dt.isocalendar().week
    df["Week_sin"] = np.sin(2 * np.pi * df["Week"])
    df["Week_cos"] = np.cos(2 * np.pi * df["Week"])
    df["Year_sin"] = np.sin(2 * np.pi * df["Year"])
    df["Year_cos"] = np.cos(2 * np.pi * df["Year"])
    df["Month_sin"] = np.sin(2 * np.pi * df["Month"] / 12)
    df["Month_cos"] = np.cos(2 * np.pi * df["Month"] / 12)
    df["Day_sin"] = np.sin(2 * np.pi * df["Day"] / 31)
    df["Day_cos"] = np.cos(2 * np.pi * df["Day"] / 31)
    df["Group"] = (df["Year"] - 2020) * 48 + df["Month"] * 4 + df["Day"] // 7
    df.drop("Policy Start Date", axis=1, inplace=True)
    return df

In [8]:
def add_new_features(df):
    df["Income to Dependents Ratio"] = df["Annual Income"] / (
        df["Number of Dependents"].fillna(0) + 1
    )
    df["Income_per_Dependent"] = df["Annual Income"] / (df["Number of Dependents"] + 1)
    df["CreditScore_InsuranceDuration"] = df["Credit Score"] * df["Insurance Duration"]
    df["Health_Risk_Score"] = (
        df["Smoking Status"].apply(lambda x: 1 if x == "Smoker" else 0)
        + df["Exercise Frequency"].apply(
            lambda x: 1 if x == "Low" else (0.5 if x == "Medium" else 0)
        )
        + (100 - df["Health Score"]) / 20
    )
    df["Credit_Health_Score"] = df["Credit Score"] * df["Health Score"]
    df["Health_Age_Interaction"] = df["Health Score"] * df["Age"]
    df["contract length"] = pd.cut(
        df["Insurance Duration"].fillna(99),
        bins=[-float("inf"), 1, 3, float("inf")],
        labels=[0, 1, 2],
    ).astype(int)
    return df

In [9]:
def frequency_encode(df):
    cat_cols = [
        "Gender",
        "Marital Status",
        "Education Level",
        "Occupation",
        "Location",
        "Policy Type",
        "Customer Feedback",
        "Smoking Status",
        "Exercise Frequency",
        "Property Type",
    ]
    for col in cat_cols:
        freq_encoding = df[col].value_counts().to_dict()
        new_col_name = f"{col}_freq"
        df[new_col_name] = df[col].map(freq_encoding).astype("float")
        df.drop(columns=[col], inplace=True)
    return df

In [10]:
add_new_num_features = FunctionTransformer(func=add_new_features)
add_new_date_features = FunctionTransformer(func=date)
add_new_freq_features = FunctionTransformer(func=frequency_encode)

preprocessor = ColumnTransformer(
    [
        ("date_features", add_new_date_features, ["Policy Start Date"]),
        (
            "num_features",
            add_new_num_features,
            [
                "Annual Income",
                "Number of Dependents",
                "Credit Score",
                "Insurance Duration",
                "Smoking Status",
                "Exercise Frequency",
                "Health Score",
                "Age",
            ],
        ),
        (
            "freq_features",
            add_new_freq_features,
            [
                "Gender",
                "Marital Status",
                "Education Level",
                "Occupation",
                "Location",
                "Policy Type",
                "Customer Feedback",
                "Smoking Status",
                "Exercise Frequency",
                "Property Type",
                # "Month_name",
                # "Day_of_week",
            ],
        ),
        ("dropping", "drop", ["id"]),
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
    n_jobs=-1,
).set_output(transform="pandas")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_function_transformer.py:343: UserWarning: With transform="pandas", `func` should return a DataFrame to follow the set_output API.
  warnings.warn(


In [11]:
df = preprocessor.fit_transform(df)
df["Premium Amount"] = np.log1p(df["Premium Amount"])

In [12]:
df

,Year,Day,Month,Month_name,Day_of_week,Week,Week_sin,Week_cos,Year_sin,Year_cos,...,Occupation_freq,Location_freq,Policy Type_freq,Customer Feedback_freq,Smoking Status_freq,Exercise Frequency_freq,Property Type_freq,Previous Claims,Vehicle Age,Premium Amount
0,2023,23,12,December,Saturday,51,-0.0,1.0,-6.447061e-13,1.0,...,282645.0,397511.0,401846.0,375518.0,598127.0,306179.0,400349.0,2.0,17.0,7.962067
1,2023,12,6,June,Monday,24,-0.0,1.0,-6.447061e-13,1.0,...,NaN,400947.0,399600.0,377905.0,601873.0,299830.0,400349.0,1.0,12.0,7.302496
2,2023,30,9,September,Saturday,39,-0.0,1.0,-6.447061e-13,1.0,...,282645.0,401542.0,401846.0,368753.0,601873.0,306179.0,400349.0,1.0,14.0,6.342121
3,2024,12,6,June,Wednesday,24,-0.0,1.0,1.585375e-14,1.0,...,NaN,400947.0,398554.0,375518.0,601873.0,294571.0,399978.0,1.0,0.0,6.641182
4,2021,1,12,December,Wednesday,48,-0.0,1.0,-1.468363e-13,1.0,...,282645.0,400947.0,401846.0,375518.0,601873.0,306179.0,400349.0,0.0,8.0,7.612337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,2023,3,5,May,Wednesday,18,-0.0,1.0,-6.447061e-13,1.0,...,276530.0,397511.0,401846.0,375518.0,598127.0,294571.0,399978.0,NaN,5.0,7.173192
1199996,2022,10,9,September,Saturday,36,-0.0,1.0,-1.305266e-12,1.0,...,282645.0,400947.0,399600.0,375518.0,598127.0,306179.0,399978.0,NaN,10.0,6.711740
1199997,2021,25,5,May,Tuesday,21,0.0,1.0,-1.468363e-13,1.0,...,NaN,401542.0,398554.0,368753.0,598127.0,299830.0,399673.0,0.0,19.0,5.918894
1199998,2021,19,9,September,Sunday,37,-0.0,1.0,-1.468363e-13,1.0,...,NaN,401542.0,401846.0,375518.0,598127.0,294571.0,399978.0,1.0,7.0,6.391917


In [13]:
train_data, test_data = train_test_split(df, test_size=0.2)

In [14]:
task = Task("reg", metric="rmsle")

In [15]:
roles = {
    "target": "Premium Amount",
}

In [16]:
automl = TabularAutoML(
    task=task,
    timeout=3600,
    cpu_limit=4,
    gpu_ids='all',
    reader_params={"n_jobs": 4, "cv": 5},
)

In [17]:
out_of_fold_predictions = automl.fit_predict(train_data, roles=roles, verbose=1)

[21:29:51] Stdout logging level is INFO.
[21:29:51] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[21:29:51] Task: reg

[21:29:51] Start automl preset with listed constraints:
[21:29:51] - time: 3600.00 seconds
[21:29:51] - CPU: 4 cores
[21:29:51] - memory: 16 GB

[21:29:51] Train data shape: (960000, 43)

[21:30:04] Layer 1 train process start. Time left 3586.53 secs


/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,


[21:30:41] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[21:31:48] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -0.16325699286287418
[21:31:48] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[21:31:48] Time left 3482.95 secs

[21:32:07] Selector_LightGBM fitting and predicting completed
[21:32:28] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[21:33:54] Fitting Lvl_0_Pipe_1_Mod_0_LightGBM finished. score = -0.1576349802394631
[21:33:54] Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predicting completed
[21:33:54] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ... Time budget is 300.00 secs


Optimization Progress:  18%|█▊        | 18/101 [05:17<24:25, 17.66s/it, best_trial=11, best_value=-0.157]

[21:39:12] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM completed
[21:39:12] Start fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ...


[21:40:43] Fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM finished. score = -0.15761263895700214
[21:40:43] Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM fitting and predicting completed
[21:40:43] Start fitting Lvl_0_Pipe_1_Mod_2_CatBoost ...
[21:42:47] Fitting Lvl_0_Pipe_1_Mod_2_CatBoost finished. score = -0.15815840222975486
[21:42:47] Lvl_0_Pipe_1_Mod_2_CatBoost fitting and predicting completed
[21:42:47] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ... Time budget is 300.00 secs


Optimization Progress:  16%|█▌        | 16/101 [05:08<27:21, 19.31s/it, best_trial=15, best_value=-0.158]

[21:47:57] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost completed
[21:47:57] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...


[21:49:25] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = -0.15809718083020322
[21:49:25] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[21:49:25] Time left 2425.64 secs

[21:49:25] Layer 1 training completed.

[21:49:25] Blending: optimization starts with equal weights. Score = -0.1581343
[21:49:27] Blending: iteration 0: score = -0.1576006, weights = [0.         0.37146237 0.6285376  0.         0.        ]
[21:49:29] Blending: no improvements for score. Terminated.

[21:49:29] Blending: best score = -0.1576006, best weights = [0.         0.37146237 0.6285376  0.         0.        ]
[21:49:29] Automl preset training completed in 1177.78 seconds

[21:49:29] Model description:
Final prediction for new objects (level 0) = 
	 0.37146 * (5 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.62854 * (5 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) 



In [18]:
predictions = automl.predict(test_data)

In [19]:
# Предсказания с преобразованием в одномерный массив
predictions = predictions.data
predictions = predictions.ravel()

In [20]:
np.sqrt(mean_squared_log_error(
    np.expm1(test_data["Premium Amount"]), np.expm1(predictions)
))

1.0460729634408008

1.041886366564917

In [47]:
X = pd.read_csv("/kaggle/input/compete/test.csv")

In [48]:
ids = X["id"]

In [49]:
X = preprocessor.fit_transform(X)

In [50]:
predictions_test = automl.predict(X)
predictions_test = predictions_test.data
predictions_test = predictions_test.ravel()
predictions_test = np.expm1(predictions_test)

In [51]:
predictions_test_df = pd.DataFrame(predictions_test, columns=["Premium Amount"])

result = pd.concat([ids, predictions_test_df], axis=1)

In [52]:
result.to_csv("submission.csv", index=False)